In [164]:
import pm4py

event_log = pm4py.read_xes("../Data Sets/Loans/BPI Challenge 2017.xes.gz")
start_activities = pm4py.get_start_activities(event_log)
end_activities = pm4py.get_end_activities(event_log)
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))
df = pm4py.convert_to_dataframe(event_log)
print("conversion to dataframe done")
df.to_csv('../Data Sets/Loans/BPI Challenge 2017.csv')
print("CSV created")

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

Start activities: {'A_Create Application': 31509}
End activities: {'W_Validate application': 12662, 'O_Cancelled': 4436, 'W_Call after offers': 9457, 'W_Call incomplete files': 4676, 'W_Complete application': 144, 'W_Assess potential fraud': 102, 'W_Personal Loan collection': 2, 'W_Shortened completion ': 5, 'A_Denied': 1, 'O_Sent (online only)': 8, 'O_Sent (mail and online)': 15, 'O_Returned': 1}
conversion to dataframe done
CSV created


In [165]:
import pandas as pd

event_log = pandas.read_csv("../Data Sets/Loans/BPI Challenge 2017.csv",sep=',')
event_log.drop(event_log.columns[[0]], axis=1, inplace = True) 
event_log.head()



,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
for col in event_log: print(col)
    

Action
org:resource
concept:name
EventOrigin
EventID
lifecycle:transition
time:timestamp
case:LoanGoal
case:ApplicationType
case:concept:name
case:RequestedAmount
FirstWithdrawalAmount
NumberOfTerms
Accepted
MonthlyCost
Selected
CreditScore
OfferedAmount
OfferID


get the 'time:timestamp' values as a series, to transform the values (less millisecs, no timezone). Some values were wrong (missing millisec)

In [133]:

def harmonize_the_date(thedate) :
    if (len(thedate) == 32) :
        # 2016-01-01 09:51:15.304000+00:00
        # remove the last 3 digits of the milliseconds
        return (thedate[0 : 23] + thedate[-6:])
    else :
        # 2016-01-02 14:00:20+00:00
        # remove the end and add .000 ms
        return (thedate[0 : 19] + ".000" + thedate[-6:])
    
event_log['time:timestamp'] = event_log['time:timestamp'].apply(harmonize_the_date)

event_log.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create a smaller event log to work faster and load faster in Process Mining

In [85]:
less_events = event_log[:2000]
less_events.to_csv("../Data Sets/Loans/LessEvents.csv", index=False, na_rep='' )

Declutter the event log: removing all events that are irrelevant, duplicate to other events.

lifecycle:transition: schedule. 
Created instantly after another event. What matters is when this work item is started, or completed.

Let's see what if we only keep 'complete' transitions

Supress useless events: O_Create Offer and O_created are identical 
A_Create Application and A_Submitted

In [156]:
#Remove all events with transition <> complete
complete = event_log[event_log["lifecycle:transition"].str.contains("complete")]
#events with ate_abort are never complete, so they do not appear. 

#Create a mask to remove all the events that are irrelevant
complete = complete.mask((event_log["concept:name"].str.contains("O_Created") 
                      | event_log["concept:name"].str.contains("A_Submitted")
                      | event_log["concept:name"].str.contains("W_Complete application")
                     ))
# remove empty rows (NaN)
complete = complete[complete["concept:name"].notna()]
complete.to_csv("../Data Sets/Loans/Complete2.csv", index=False, na_rep='' )
complete

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,statechange,User_52,A_Accepted,Application,ApplState_99568828,complete,2016-01-02 11:23:04.299+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Created,User_52,O_Create Offer,Offer,Offer_148581083,complete,2016-01-02 11:29:03.994+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,20000.0,44.0,True,498.29,True,979.0,20000.0,NaN
11,statechange,User_52,O_Sent (mail and online),Offer,OfferState_2051164740,complete,2016-01-02 11:30:28.606+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_148581083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202254,Created,User_96,O_Create Offer,Offer,Offer_1580299144,complete,2017-01-02 19:24:59.501+00:00,Home improvement,New credit,Application_1350494635,20000.0,20000.0,77.0,False,297.81,False,0.0,20000.0,NaN
1202256,statechange,User_96,O_Sent (mail and online),Offer,OfferState_1959881309,complete,2017-01-02 19:27:20.453+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144
1202260,statechange,User_96,A_Complete,Application,ApplState_1120616436,complete,2017-01-02 19:27:20.474+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Split the original event log into A_, W_, and O_ events


In [163]:
w_logs = event_log[event_log["concept:name"].str.startswith("W_")]
w_logs.to_csv("../Data Sets/Loans/w_events.csv")

In [159]:
A_logs = event_log[event_log["concept:name"].str.startswith("A_")]
A_logs.to_csv("../Data Sets/Loans/a_events.csv")

In [161]:
O_logs = event_log[event_log["concept:name"].str.startswith("O_")]
O_logs.to_csv("../Data Sets/Loans/o_events.csv")